### Sort Data Script 

Data under data/raw_data looks like: 
    
    raw_data ----- sample_id1 --- .tsv file 
              |___ sample_id2 --- .tsv file 
              |____ ....
              .
              .___ sample_idn --- .tsv file 

This script sorts the tsv files by storing each tsv file under their respective case ID's (i.e. patient ID). 
So the processed data files will be stored under data/processed_data like this: 

    processed_data ------ case_id1 ----- .tsv file 1 
                      |           |_____ ..... 
                      |           . 
                      |           ._____ .tsv file n 
                      |
                      |
                      |
                      |__ case_id2 ----- .tsv file 1 
                      |           |_____ ..... 
                      |           . 
                      |           ._____ .tsv file n 
                      |
                      . 
                      .
                      .___ case_idx ----- .tsv file 1 
                                  |_____ ..... 
                                  . 
                                  ._____ .tsv file n 



NOTE: This script queries GDC Endpoint for Case ID
                      


In [1]:
import pandas as pd
import requests
import os
from tqdm import tqdm
import shutil

In [3]:
# GET FOLDER NAMES (SAMPLE ID'S)
def get_folder_names(current_directory):
    items_in_directory = os.listdir(current_directory)
    directories_in_current_directory = [item for item in items_in_directory if os.path.isdir(os.path.join(current_directory, item))]
    return directories_in_current_directory

sample_ids = get_folder_names('../data/raw_data/')
sample_ids


['5158a1a5-4505-46da-abaf-ad97b93ad9c3',
 '251ac37b-69a5-4866-ac5b-be853d5e4314',
 '88962ea5-7761-4a8f-bba6-3a63662af1a3',
 'e87863d9-e00a-44d8-8331-bb9214b5554a',
 '985cc5cd-d610-44e2-95b5-6844ccd67306',
 'e41bd510-0f59-438a-8db2-cf6042032329',
 '0b905aa8-6ef7-490c-8a7c-a64694b9f513',
 'dacd59de-588e-49be-b0a2-a2e4550f0d57',
 'b6a17c1d-7f72-412b-8e55-fa8569c0ef18',
 'e3b25001-4358-4537-aa2a-2ee9ff9ca4e7',
 '1c65a914-ff06-4103-835c-47e62ab74517',
 'a3d030a7-9b57-48de-a156-3d3bccaaa840',
 'fbb80db9-04d6-4cea-838c-f5ed7523cbdd',
 'f3e67e23-444d-44db-ad15-c6b05250ac32',
 '266fd576-2577-4294-bdaf-2d2d34a272b4',
 '9db51366-c3ea-4510-904f-883f1a316cfe',
 'e4dbc1f5-1dc2-450b-a1c6-2218d3f5cb74',
 '61772045-9b3a-4c36-8963-83a0b0906140',
 'f8ce8b87-c50b-483c-a5ad-83ae08799e17',
 '3fc85921-05d1-4259-89fe-b76b081e6ee7',
 '791bbb6f-def9-47aa-b189-297205b68d99',
 'b9e86156-9e09-483b-88e5-38da3af2fe65',
 '8aad5b2e-b6fe-4f71-ba95-bd03e4354aaf',
 '8c84e99a-c853-4cb6-8b7f-8293d174f515',
 '1ff72926-3b7a-

In [4]:
# FUNCTION TO GET CASE ID ASSOCIATED WITH SAMPLE ID 
def get_case_id(file_id):
    # Construct the API request URL to search for cases containing the file
    api_url = "https://api.gdc.cancer.gov/cases"

    # Define the query payload to search for cases containing the file
    query = {
        "filters": {
            "op": "=",
            "content": {
                "field": "files.file_id",
                "value": [file_id]
            }
        },
        "fields": "case_id"
    }

    # Send a POST request to the GDC API to search for cases containing the file
    response = requests.post(api_url, json=query)

    if response.status_code == 200:
        data = response.json()["data"]

        if data["pagination"]["total"] > 0:
            # Extracting the case UUID(s) associated with the file
            case_uuids = [case["case_id"] for case in data["hits"]]
            return case_uuids[0] # only return first case id found
        else:
            print(f"No cases found containing file {file_id}")
    else:
        print(f"Failed to retrieve information for file {file_id}")


# ORGANIZE SAMPLE IDS UNDER CASE ID
case_to_sample_id = {}
for sample_id in tqdm(sample_ids): 
    case_id = get_case_id(sample_id)

    if case_id in case_to_sample_id: 
        case_to_sample_id[case_id].append(sample_id)
    else:
        case_to_sample_id[case_id] = [sample_id]

case_to_sample_id

100%|██████████| 500/500 [02:58<00:00,  2.80it/s]


{'0ec70e40-07df-461d-8ff9-351240a0d454': ['5158a1a5-4505-46da-abaf-ad97b93ad9c3'],
 'dc696e3c-f448-468f-a576-f4429be0338a': ['251ac37b-69a5-4866-ac5b-be853d5e4314',
  'ec1b18ca-ea61-44f3-88fc-86248e9a0640'],
 '2bcacfd1-6a45-4b96-9b80-2ad569b1ab45': ['88962ea5-7761-4a8f-bba6-3a63662af1a3',
  '229219e2-9b56-4882-8741-cee697c4b714'],
 '074e8b06-3929-4896-9263-3b80ad6a355b': ['e87863d9-e00a-44d8-8331-bb9214b5554a'],
 'dcb68327-1537-4ca6-81de-3910d0663cb8': ['985cc5cd-d610-44e2-95b5-6844ccd67306',
  '27b27f78-db38-4044-a71d-4969cef96207'],
 'dfaabd03-2d40-4422-b210-caf112ff4229': ['e41bd510-0f59-438a-8db2-cf6042032329',
  '7dbd1a51-3b6c-47b9-a608-2a010e0149ca'],
 '1d38d356-d126-4476-94d0-26616b9375b1': ['0b905aa8-6ef7-490c-8a7c-a64694b9f513'],
 '10c9be1e-2cc0-45e4-8d2c-c013ea63ef05': ['dacd59de-588e-49be-b0a2-a2e4550f0d57'],
 '518f231c-f5f9-4a39-a97b-7fd5666d3e50': ['b6a17c1d-7f72-412b-8e55-fa8569c0ef18'],
 '59be60a3-e0f6-4d63-b98c-574ef92428c4': ['e3b25001-4358-4537-aa2a-2ee9ff9ca4e7'],
 '

In [5]:
#### FUNCTION: COPYS OVER ALL TSV FILES FOR GIVEN CASE ID 
def copy_all_files(case_id, case_to_sample_id): 
    for sample_id in case_to_sample_id[case_id]:

        file_name = [f for f in os.listdir("../data/raw_data/" + str(sample_id)) if f.endswith(".tsv")][0]

        if file_name.endswith(".txt"): 
            print(f"ERROR: {file_name} found in folder {sample_id}")

        source_path = "../data/raw_data/" + str(sample_id) + "/" + file_name
        target_path = "../data/processed_data/" + str(case_id) + "/" + file_name
        
        os.makedirs("../data/processed_data/" + str(case_id), exist_ok=True)
        shutil.copy(source_path, target_path)

# ITERATE THRU DICTIONARY TO COPY ALL FILES
for case_id in tqdm(case_to_sample_id.keys()):
    copy_all_files(case_id, case_to_sample_id)


100%|██████████| 319/319 [00:05<00:00, 58.72it/s]
